In [1]:
!gdown 1nQDUEBgCF8087nPuLxNfBgJFeaJPaWoq
# Configure Kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the Retinal Disease Classification dataset
!kaggle datasets download -d johnsmith88/heart-disease-dataset

Downloading...
From: https://drive.google.com/uc?id=1nQDUEBgCF8087nPuLxNfBgJFeaJPaWoq
To: /content/kaggle.json
100% 69.0/69.0 [00:00<00:00, 231kB/s]
Dataset URL: https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset
License(s): unknown
  0% 0.00/6.18k [00:00<?, ?B/s]
100% 6.18k/6.18k [00:00<00:00, 2.62MB/s]


In [2]:

!unzip -qq heart-disease-dataset -d data
!rm -rf heart-disease-dataset.zip

In [3]:
import plotly.express as px
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

In [6]:
import pandas as pd

# Load the dataset
data_path = "data/heart.csv"
df = pd.read_csv(data_path)

# Inspect the dataset
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 5   fbs       1025 non-null   int64  
 6   restecg   1025 non-null   int64  
 7   thalach   1025 non-null   int64  
 8   exang     1025 non-null   int64  
 9   oldpeak   1025 non-null   float64
 10  slope     1025 non-null   int64  
 11  ca        1025 non-null   int64  
 12  thal      1025 non-null   int64  
 13  target    1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB


In [8]:
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,1025.000000,1025.000000,1025.000000,1025.000000,1025.00000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000
mean,54.434146,0.695610,0.942439,131.611707,246.00000,0.149268,0.529756,149.114146,0.336585,1.071512,1.385366,0.754146,2.323902,0.513171
std,9.072290,0.460373,1.029641,17.516718,51.59251,0.356527,0.527878,23.005724,0.472772,1.175053,0.617755,1.030798,0.620660,0.500070
min,29.000000,0.000000,0.000000,94.000000,126.00000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,48.000000,0.000000,0.000000,120.000000,211.00000,0.000000,0.000000,132.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,56.000000,1.000000,1.000000,130.000000,240.00000,0.000000,1.000000,152.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,275.00000,0.000000,1.000000,166.000000,1.000000,1.800000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.00000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [9]:
from sklearn.preprocessing import StandardScaler

# Check for missing values
print("\nMissing Values:")
print(df.isnull().sum())

# Separate features and labels
X = df.drop(columns=["target"])  # Assuming 'target' is the label column
y = df["target"]  # Target labels

# Standardize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\nShape of Scaled Features:", X_scaled.shape)


Missing Values:
age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

Shape of Scaled Features: (1025, 13)


In [10]:
from sklearn.model_selection import train_test_split

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print(f"Training samples: {X_train.shape[0]}, Testing samples: {X_test.shape[0]}")


Training samples: 820, Testing samples: 205


## Step 1: Apply Locally Linear Embedding (LLE)


In [11]:
from sklearn.manifold import LocallyLinearEmbedding
import pandas as pd
import plotly.express as px

# Parameters for LLE
n_neighbors = 10  # Number of neighbors to consider
n_components = 3  # Reduce to 3 dimensions for 3D visualization

# Apply LLE
lle = LocallyLinearEmbedding(n_neighbors=n_neighbors, n_components=n_components, method='standard')
X_train_lle_3d = lle.fit_transform(X_train)
X_test_lle_3d = lle.transform(X_test)

# Create a DataFrame for visualization
lle_3d_df = pd.DataFrame(X_train_lle_3d, columns=["Component 1", "Component 2", "Component 3"])
lle_3d_df['Target'] = y_train.values  # Add target labels

# Plot the interactive 3D scatter plot
fig = px.scatter_3d(lle_3d_df,
                    x="Component 1",
                    y="Component 2",
                    z="Component 3",
                    color=lle_3d_df['Target'].astype(str),
                    title="Locally Linear Embedding (LLE) - 3D Visualization",
                    labels={"Target": "Class Label"})
fig.show()

## Step 2: Apply t-SNE


In [13]:
from sklearn.manifold import TSNE
import pandas as pd
import plotly.express as px

# Parameters for t-SNE
n_components = 3  # Reduce to 3 dimensions for 3D visualization
perplexity = 30  # Controls the balance between local and global structure
random_state = 42  # For reproducibility

# Apply t-SNE
tsne = TSNE(n_components=n_components, perplexity=perplexity, random_state=random_state)
X_train_tsne_3d = tsne.fit_transform(X_train)

# Create a DataFrame for visualization
tsne_3d_df = pd.DataFrame(X_train_tsne_3d, columns=["Component 1", "Component 2", "Component 3"])
tsne_3d_df['Target'] = y_train.values  # Add target labels

# Plot the interactive 3D scatter plot
fig = px.scatter_3d(tsne_3d_df,
                    x="Component 1",
                    y="Component 2",
                    z="Component 3",
                    color=tsne_3d_df['Target'].astype(str),
                    title="t-SNE - 3D Visualization",
                    labels={"Target": "Class Label"})
fig.show()


## Step 3: Apply ISOMAP

In [14]:
from sklearn.manifold import Isomap
import pandas as pd
import plotly.express as px

# Parameters for ISOMAP
n_neighbors = 10  # Number of neighbors to consider
n_components = 3  # Reduce to 3 dimensions for 3D visualization

# Apply ISOMAP
isomap = Isomap(n_neighbors=n_neighbors, n_components=n_components)
X_train_isomap_3d = isomap.fit_transform(X_train)

# Create a DataFrame for visualization
isomap_3d_df = pd.DataFrame(X_train_isomap_3d, columns=["Component 1", "Component 2", "Component 3"])
isomap_3d_df['Target'] = y_train.values  # Add target labels

# Plot the interactive 3D scatter plot
fig = px.scatter_3d(isomap_3d_df,
                    x="Component 1",
                    y="Component 2",
                    z="Component 3",
                    color=isomap_3d_df['Target'].astype(str),
                    title="ISOMAP - 3D Visualization",
                    labels={"Target": "Class Label"})
fig.show()


## Step 4: Apply UMAP


In [16]:
!pip install umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.1 MB/s eta 0:00:00


In [18]:
import umap.umap_ as umap
import pandas as pd
import plotly.express as px

# Parameters for UMAP
n_neighbors = 15  # Number of neighbors to consider
min_dist = 0.1  # Minimum distance between points in the low-dimensional space
n_components = 3  # Reduce to 3 dimensions for 3D visualization

# Apply UMAP
umap_reducer = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components, random_state=42)
X_train_umap_3d = umap_reducer.fit_transform(X_train)

# Create a DataFrame for visualization
umap_3d_df = pd.DataFrame(X_train_umap_3d, columns=["Component 1", "Component 2", "Component 3"])
umap_3d_df['Target'] = y_train.values  # Add target labels

# Plot the interactive 3D scatter plot
fig = px.scatter_3d(umap_3d_df,
                    x="Component 1",
                    y="Component 2",
                    z="Component 3",
                    color=umap_3d_df['Target'].astype(str),
                    title="UMAP - 3D Visualization",
                    labels={"Target": "Class Label"})
fig.show()

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



## Step 5: Apply MDS


In [19]:
from sklearn.manifold import MDS
import pandas as pd
import plotly.express as px

# Parameters for MDS
n_components = 3  # Reduce to 3 dimensions for 3D visualization
random_state = 42  # For reproducibility

# Apply MDS
mds = MDS(n_components=n_components, random_state=random_state)
X_train_mds_3d = mds.fit_transform(X_train)

# Create a DataFrame for visualization
mds_3d_df = pd.DataFrame(X_train_mds_3d, columns=["Component 1", "Component 2", "Component 3"])
mds_3d_df['Target'] = y_train.values  # Add target labels

# Plot the interactive 3D scatter plot
fig = px.scatter_3d(mds_3d_df,
                    x="Component 1",
                    y="Component 2",
                    z="Component 3",
                    color=mds_3d_df['Target'].astype(str),
                    title="MDS - 3D Visualization",
                    labels={"Target": "Class Label"})
fig.show()


## Step 6: Apply Randomized PCA

In [20]:
from sklearn.decomposition import PCA
import pandas as pd
import plotly.express as px

# Parameters for Randomized PCA
n_components = 3  # Reduce to 3 dimensions for 3D visualization
random_state = 42  # For reproducibility

# Apply Randomized PCA
randomized_pca = PCA(n_components=n_components, svd_solver='randomized', random_state=random_state)
X_train_rpca_3d = randomized_pca.fit_transform(X_train)

# Create a DataFrame for visualization
rpca_3d_df = pd.DataFrame(X_train_rpca_3d, columns=["Component 1", "Component 2", "Component 3"])
rpca_3d_df['Target'] = y_train.values  # Add target labels

# Plot the interactive 3D scatter plot
fig = px.scatter_3d(rpca_3d_df,
                    x="Component 1",
                    y="Component 2",
                    z="Component 3",
                    color=rpca_3d_df['Target'].astype(str),
                    title="Randomized PCA - 3D Visualization",
                    labels={"Target": "Class Label"})
fig.show()

## Step 7: Apply Kernel PCA


In [21]:
from sklearn.decomposition import KernelPCA
import pandas as pd
import plotly.express as px

# Parameters for Kernel PCA
n_components = 3  # Reduce to 3 dimensions for 3D visualization
kernel = 'rbf'  # Radial basis function kernel
gamma = 0.1  # Kernel coefficient for RBF kernel

# Apply Kernel PCA
kernel_pca = KernelPCA(n_components=n_components, kernel=kernel, gamma=gamma, random_state=42)
X_train_kpca_3d = kernel_pca.fit_transform(X_train)

# Create a DataFrame for visualization
kpca_3d_df = pd.DataFrame(X_train_kpca_3d, columns=["Component 1", "Component 2", "Component 3"])
kpca_3d_df['Target'] = y_train.values  # Add target labels

# Plot the interactive 3D scatter plot
fig = px.scatter_3d(kpca_3d_df,
                    x="Component 1",
                    y="Component 2",
                    z="Component 3",
                    color=kpca_3d_df['Target'].astype(str),
                    title="Kernel PCA (RBF Kernel) - 3D Visualization",
                    labels={"Target": "Class Label"})
fig.show()


## Step 8: Apply Incremental PCA



In [22]:
from sklearn.decomposition import IncrementalPCA
import pandas as pd
import plotly.express as px

# Parameters for Incremental PCA
n_components = 3  # Reduce to 3 dimensions for 3D visualization
batch_size = 100  # Process data in chunks

# Apply Incremental PCA
incremental_pca = IncrementalPCA(n_components=n_components, batch_size=batch_size)
X_train_ipca_3d = incremental_pca.fit_transform(X_train)

# Create a DataFrame for visualization
ipca_3d_df = pd.DataFrame(X_train_ipca_3d, columns=["Component 1", "Component 2", "Component 3"])
ipca_3d_df['Target'] = y_train.values  # Add target labels

# Plot the interactive 3D scatter plot
fig = px.scatter_3d(ipca_3d_df,
                    x="Component 1",
                    y="Component 2",
                    z="Component 3",
                    color=ipca_3d_df['Target'].astype(str),
                    title="Incremental PCA - 3D Visualization",
                    labels={"Target": "Class Label"})
fig.show()


## Step 9: Apply Factor Analysis


In [23]:
from sklearn.decomposition import FactorAnalysis
import pandas as pd
import plotly.express as px

# Parameters for Factor Analysis
n_components = 3  # Reduce to 3 dimensions for 3D visualization

# Apply Factor Analysis
factor_analysis = FactorAnalysis(n_components=n_components, random_state=42)
X_train_fa_3d = factor_analysis.fit_transform(X_train)

# Create a DataFrame for visualization
fa_3d_df = pd.DataFrame(X_train_fa_3d, columns=["Component 1", "Component 2", "Component 3"])
fa_3d_df['Target'] = y_train.values  # Add target labels

# Plot the interactive 3D scatter plot
fig = px.scatter_3d(fa_3d_df,
                    x="Component 1",
                    y="Component 2",
                    z="Component 3",
                    color=fa_3d_df['Target'].astype(str),
                    title="Factor Analysis - 3D Visualization",
                    labels={"Target": "Class Label"})
fig.show()


## Step 10: Apply Autoencoders


In [24]:
!pip install tensorflow --quiet

In [25]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
import plotly.express as px

# Parameters for Autoencoder
input_dim = X_train.shape[1]  # Number of features
encoding_dim = 3  # Reduce to 3 dimensions for visualization

# Build the Autoencoder model
input_layer = layers.Input(shape=(input_dim,))
encoded = layers.Dense(encoding_dim, activation='relu')(input_layer)
decoded = layers.Dense(input_dim, activation='sigmoid')(encoded)

autoencoder = models.Model(input_layer, decoded)
encoder = models.Model(input_layer, encoded)

# Compile the Autoencoder
autoencoder.compile(optimizer='adam', loss='mse')

# Train the Autoencoder
autoencoder.fit(X_train, X_train,
                epochs=50,
                batch_size=32,
                shuffle=True,
                validation_split=0.2,
                verbose=1)

# Use the encoder to reduce dimensionality
X_train_autoencoded_3d = encoder.predict(X_train)

# Create a DataFrame for visualization
autoencoded_3d_df = pd.DataFrame(X_train_autoencoded_3d, columns=["Component 1", "Component 2", "Component 3"])
autoencoded_3d_df['Target'] = y_train.values  # Add target labels

# Plot the interactive 3D scatter plot
fig = px.scatter_3d(autoencoded_3d_df,
                    x="Component 1",
                    y="Component 2",
                    z="Component 3",
                    color=autoencoded_3d_df['Target'].astype(str),
                    title="Autoencoders - 3D Visualization",
                    labels={"Target": "Class Label"})
fig.show()


Epoch 1/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - loss: 1.2447 - val_loss: 1.2473
Epoch 2/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 1.2496 - val_loss: 1.2326
Epoch 3/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.2370 - val_loss: 1.2181
Epoch 4/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.2077 - val_loss: 1.2039
Epoch 5/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.1983 - val_loss: 1.1892
Epoch 6/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.1577 - val_loss: 1.1747
Epoch 7/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 1.1663 - val_loss: 1.1600
Epoch 8/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1.1248 - val_loss: 1.1452
Epoch 9/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 1.1286 - val_loss: 1.1311
Epoch 10/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.1251 - val_loss: 1.1172
Epoch 11/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.0997 - val_loss: 1.1044
Epoch 12/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.075